<a href="https://colab.research.google.com/github/mahdi943/colsest-capitals/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Using Capitals.txt dataset;
Find two closest capital cities and the distance between them.

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In the first step, we need removing double space characters as well as changing the ',' character in the float numbers to '.' in Capitals.txt

In [ ]:
myFile = open('/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Cc.txt', 'w')
with open('/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Capitals.txt', 'r') as f:
  for line in f:
    st = line.replace(' ', '').replace(',','.')
    myFile.write(st)
f.close()
myFile.close()

Since  "Jerusalem" and "EastJerusalem" are same,
also "Rome" and "Vatican" are both in Italy and "Rome" is the capital I filtered them.



In [ ]:
import math
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

capRDD = sc.textFile("/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Cc.txt")
def saveCoord(line):
    rec = line.split("\t")
    name = rec[2]
    lat = float(rec[3])
    lng = float(rec[4])
    return(name,lat,lng)
    
capRDD1 = capRDD.map(lambda x : saveCoord(x) )
capRDD2 = capRDD.map(lambda x : saveCoord(x) )

capRDD3 = capRDD1.cartesian(capRDD2)
capRDD3 = capRDD3.map(lambda x : x[0]+x[1])



def distance(city1, city2, lat1, lon1, lat2, lon2):
  if city1 != city2:
    radius = 6371  # km
    lat1 = float(lat1)
    lon1 = float(lon1)
    lat2 = float(lat2)
    lon2 = float(lon2)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
      math.cos(math.radians(lat1)) *
      math.cos(math.radians(lat2)) * math.sin(dlon / 2) *
      math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return (city1,city2,d)

capRDD3 = capRDD3.map(lambda x : distance(x[0],x[3],x[1],x[2],x[4],x[5]))
#print(capRDD3.take(10))

# since we applide cartesian, we should apply filter for both 
capRDD3  = capRDD3.filter(lambda x: x is not None)
capRDD3 = capRDD3.filter(lambda x: x[1] != "Vatican" and  x[2] != "Rome")
capRDD3 = capRDD3.filter(lambda x: x[1] != "Rome" and  x[2] != "Vatican")
capRDD3 = capRDD3.filter(lambda x: x[1] != "EastJerusalem" and  x[2] != "Jerusalem")
capRDD3 = capRDD3.filter(lambda x: x[1] != "Jerusalem" and  x[2] != "EastJerusalem")

capRDD3 = capRDD3.sortBy(lambda x : x[2],ascending=True)

print("Two closest capital cities in the world:",capRDD3.take(1))


Two closest capital cities in the world: [('Kinshasa', 'Brazzaville', 6.478907732623775)]


Two closest capital cities in the world:
"Kinshasa" and "Brazzaville" 

Distance:
6.478907732623725 Km